In [1]:
# from dotenv import load_dotenv
# import os
# load_dotenv()
# HF_TOKEN = os.getenv("HF_TOKEN")

import ssl
ssl._create_default_https_context = ssl._create_unverified_context


# ─── Prepend Homebrew to PATH ──────────────────────────────────────────────────
# so that subprocesses can find /opt/homebrew/bin/ffmpeg, etc.
import os
homebrew_bin = "/opt/homebrew/bin"
os.environ["PATH"] = f"{homebrew_bin}:{os.environ.get('PATH', '')}"


In [2]:
import psutil, time, functools, os
from collections import namedtuple

# A small helper to snapshot stats
Snapshot = namedtuple("Snapshot", ["ts", "cpu_user", "cpu_sys", "rss", "vms", "read_bytes", "write_bytes"])

def take_snapshot():
    p = psutil.Process(os.getpid())
    io = psutil.disk_io_counters()
    cpu_times = p.cpu_times()
    mem = p.memory_info()
    return Snapshot(
        ts=time.time(),
        cpu_user=cpu_times.user,
        cpu_sys=cpu_times.system,
        rss=mem.rss,
        vms=mem.vms,
        read_bytes=io.read_bytes,
        write_bytes=io.write_bytes,
    )

def profile_resources(func):
    """Decorator to measure resources used by a single call to func(...)."""
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        before = take_snapshot()
        result = func(*args, **kwargs)
        after = take_snapshot()
        # compute deltas
        wall  = after.ts - before.ts
        cpu   = (after.cpu_user + after.cpu_sys) - (before.cpu_user + before.cpu_sys)
        rss   = (after.rss - before.rss) / (1024**2)        # MB
        vms   = (after.vms - before.vms) / (1024**2)        # MB
        read  = (after.read_bytes - before.read_bytes) / (1024**2)  # MB
        write = (after.write_bytes - before.write_bytes) / (1024**2) # MB
        print(f"\n🔍 Resource profile for `{func.__name__}`:")
        print(f"  Wall-time:   {wall:.1f}s")
        print(f"  CPU time:    {cpu:.1f}s")
        print(f"  RSS Δ:       {rss:.1f} MB")
        print(f"  VMS Δ:       {vms:.1f} MB")
        print(f"  Disk Read:   {read:.1f} MB")
        print(f"  Disk Write:  {write:.1f} MB\n")
        return result
    return wrapper


In [20]:
import whisperx
from tools.comparison import get_hypothesis_text, load_reference_text, compare_texts
import gc
from tools.persian_normalizer import persian_normalizer
from hazm import Normalizer as HazmNormalizer

def cleaning(text, language=None):
    if not isinstance(text, str):
        return None

    if language == "fa":
        p_normalizer = persian_normalizer({"sentence": text}, return_dict=False)
        print("\np_normalizer\n", p_normalizer)
        h_normalizer = HazmNormalizer()
        hazm_out = h_normalizer.normalize(p_normalizer)
        print("\nhazm_out\n", hazm_out)
        return hazm_out



def segments_comparison(segments,
    ground_truth_path: str,
    audio_path: str,
    diff: bool      = False,
    print_hype_text: bool     = False,
    print_ref_text: bool      = False,
    msg: str = None,
    lang: str = None,
    ):

    # 2. hypothesis text
    hyp_text = get_hypothesis_text(segments).strip()
    if print_hype_text:
        print("\nhyp_text:\n", hyp_text)



    # 3. reference text
    ref_text = load_reference_text(ground_truth_path).strip()
    if print_ref_text:
        print("\nref_text:\n", ref_text)


    hyp = cleaning(hyp_text, lang).strip() if isinstance(hyp_text, str) else hyp_text
    # ref = cleaning(ref_text, lang).strip() if isinstance(ref_text, str) else ref_text



    # 4. compare & report
    print("\n" + "*"*70)
    print(f"Comparing {msg}\n  HYP: {audio_path}\n  REF: {ground_truth_path}")
    print("*"*70 + "\n")
    compare_texts(hyp_text, ref_text, diff=diff)

    # 4. compare & report
    print("\n" + "*"*70)
    print(f"Comparing After Normalizing{msg}\n  HYP: {audio_path}\n  REF: {ground_truth_path}")
    print("*"*70 + "\n")

    if print_hype_text:
        print("\nhyp_text after normalization:\n", hyp)

    if print_ref_text:
        print("\nref_text:\n", ref_text)

    compare_texts(hyp, ref_text, diff=diff)



def evaluate_transcription(
    audio_path: str,
    ground_truth_path: str,
    model_size: str = "small",
    device: str     = "cpu",
    compute_type: str = "float32",
    batch_size: int = 16,
    diff: bool      = False,
    print_hype_text: bool     = False,
    print_ref_text: bool      = False,
):
    """
    1) Load Whisper ASR, transcribe `audio_path` → segments
    2) Build hypothesis text
    3) Load reference text from ground_truth_path
    4) Compute & print WER (+ optional diff)
    """
    # 1. load & transcribe
    model = whisperx.load_model(model_size, device, compute_type=compute_type)
    audio = whisperx.load_audio(audio_path)
    result = model.transcribe(audio, batch_size=batch_size)
    segments_comparison(result["segments"], ground_truth_path, audio_path, diff=diff, print_hype_text=print_hype_text, print_ref_text=print_ref_text, msg="transcription", lang=result["language"])

    return result


def align_whisper_output(
    result,
    audio_path: str,
    ground_truth_path: str,
    device: str     = "cpu",
    diff: bool      = False,
    print_hype_text: bool     = False,
    print_ref_text: bool      = False,
):

    audio = whisperx.load_audio(audio_path)

    # 2. Align whisper output
    model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device, )
    result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

    segments_comparison(result["segments"], ground_truth_path, audio_path, diff=diff, print_hype_text=print_hype_text, print_ref_text=print_ref_text, msg="aligned", lang=result["language"])

In [21]:

# result = evaluate_transcription(audio_path=audio_file, ground_truth_path=ground_truth_file, diff=False, print_hype_text=True , print_ref_text=True, model_size="medium" )

@profile_resources
def run_test():

    audio_file = "../Data/Training/Farsi/Quran/farsi_secondsource.mp3"
    ground_truth_file = "../Data/Training/Farsi/Quran/farsi_secondsource_transcript-farsi_translation-hebrew.docx"

    # audio_file = "../Data/Training/Farsi/Quran/farsi_thirdsource.mp3"
    # ground_truth_file = "../Data/Training/Farsi/Quran/farsi_thirdsource_transcript-farsi_translation-hebrew.docx"

    # audio_file = "../Data/Training/Hebrew/hebrew_firstsource.mp4"
    # ground_truth_file = "../Data/Training/Hebrew/hebrew_firstsource_transcript-hebrew_translation-english.docx"

    return evaluate_transcription(
        audio_path=audio_file,
        ground_truth_path=ground_truth_file,
        diff=False,
        print_hype_text=True,
        print_ref_text=True,
        model_size="medium"
    )

# Now execute:
run_test()

# model medium
##only hazm 33%
##only p_norm 30%
##both 29.23%



# align_whisper_output(result=result, audio_path=audio_file, ground_truth_path=ground_truth_file, diff=True, print_hype_text=True , print_ref_text=True )


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../.venv/lib/python3.11/site-packages/whisperx/assets/pytorch_model.bin`


No language specified, language will be first be detected for each audio file (increases inference time).
>>Performing voice activity detection using Pyannote...
Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.7.1. Bad things might happen unless you revert torch to 1.x.
Detected language: fa (0.98) in first 30s of audio...

hyp_text:
 به نام خداوند بخشنلی بخشایشگر سوگند به کوه تور و کتابی که نوشته شده در صفحه گسترده و سوگند به بیتون معمور و سقف برپراشته و دریاج مملوغ و برپروخته که عذاب پروردارت واقع می شود و چیزی ازن مانع نخواهد بود. این عذاب الاهی در آن روزی ایس چه آسمان به شدت به حرکت درمیادید. و کوها از جا کنده و متحرک میشن. وای در آن روز بر تکزیب کنندگان. هم آنها که در صخنان باطل به بازی مشکولند. در آن روز شا آنها را به زور به سوی آتش دوزخ نیرانند. و آنها میگویند این همان آتشیست که آن را انکار میکردید. آیا این سهرست یا شما نمیدینی در آن وارد شوید و بسو

KeyboardInterrupt: 

Help on function persian_normalizer in module tools.persian_normalizer:

persian_normalizer(batch, is_normalize=True, return_dict=True, filter_trivials=False, remove_extra_space=False)

